<a href="https://colab.research.google.com/github/Shahid0120/atlassian-allianz-neural-network/blob/main/experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Neural Network - with Relu Activation Experiment with Dataloader

In [ ]:
try:
  import numpy as np
  import scipy
  import matplotlib.pyplot as plt
  import pandas as pd
  import torch
  from torch import nn
  from pathlib import Path
  from torch.optim.sgd import SGD
  from google.colab import userdata

  # Set up repo
  password_github = userdata.get('password-github')

  !git config --global user.name "Shahid0120"
  !git config --global user.email "shahid.hussain0120@gmail.com"
  !git config --global user.password password_github

  token = userdata.get('github-token')
  username = 'Shahid0120'
  repo='atlassian-allianz-neural-network'

  !git clone https://{token}@github.com/{username}/{repo}

except:
  print("Something didnt import or access not granted")


fatal: destination path 'atlassian-allianz-neural-network' already exists and is not an empty directory.


# Import dataset

In [ ]:
from pathlib import Path

dataPath = './atlassian-allianz-neural-network/data/01_encoded_no_transformations/01_encoded_no_transformations.csv'
data = pd.read_csv(dataPath)

data

,months_as_customer,age,policy_deductable,policy_annual_premium,umbrella_limit,capital-gains,incident_severity,incident_hour_of_the_day,number_of_vehicles_involved,bodily_injuries,...,auto_model_Pathfinder,auto_model_RAM,auto_model_RSX,auto_model_Silverado,auto_model_TL,auto_model_Tahoe,auto_model_Ultima,auto_model_Wrangler,auto_model_X5,auto_model_X6
0,0.556485,0.466667,0.000000,0.447388,0.545455,0.000000,0.000000,0.217391,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.671548,0.555556,0.333333,0.447797,0.090909,0.496517,0.666667,0.869565,0.666667,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.539749,0.577778,1.000000,0.459492,0.090909,0.299502,0.333333,1.000000,0.000000,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.930962,0.933333,0.333333,0.495249,0.090909,0.496517,0.333333,0.130435,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.278243,0.222222,1.000000,0.606972,0.545455,0.349254,0.333333,0.304348,0.666667,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2959,0.135983,0.155556,0.000000,0.343786,0.090909,0.000000,0.666667,0.043478,0.000000,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2960,0.364017,0.600000,0.000000,0.301668,0.090909,0.000000,0.666667,0.130435,0.666667,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2961,0.014644,0.200000,1.000000,0.510469,0.090909,0.000000,0.666667,0.869565,0.000000,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2962,0.037657,0.488889,0.000000,0.252425,0.090909,0.000000,0.333333,0.086957,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
X = data.drop(columns=['fraud_reported'])
Y = data['fraud_reported']

# Conver into tensor
X = torch.from_numpy(X.to_numpy()).type(torch.float)
y = torch.from_numpy(Y.to_numpy())

print(f"Shape of X : {X.shape}")
print(f"Shape of y : {Y.shape}")

Shape of X : torch.Size([2964, 124])
Shape of y : (2964,)


# DataLoader

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=43
)

print(f"len of X_train : {len(X_train)} | X_test : {len(X_test)} | y_train : {len(y_train)} | y_test : {len(y_test)}")


2024/09/23 23:07:00 WARNING mlflow.utils.autologging_utils: MLflow sklearn autologging is known to be compatible with 0.24.1 <= scikit-learn <= 1.5.1, but the installed version is 1.5.2. If you encounter errors during autologging, try upgrading / downgrading scikit-learn to a compatible version, or try upgrading MLflow.
2024/09/23 23:07:02 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.


len of X_train : 2371 | X_test : 593 | y_train : 2371 | y_test : 593


In [ ]:
from torch.utils.data import DataLoader, Dataset
import torch

class CustomerDataset(Dataset):
    def __init__(self, features: torch.Tensor, labels: torch.Tensor):
        self.features = features
        self.labels = labels

    def __getitem__(self, idx):
        # Return feature and label
        feature = torch.tensor(self.features[idx], dtype=torch.float32)
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        return feature, label

    def __len__(self):
        return len(self.features)

# Create dataset instances
train_dataset = CustomerDataset(X_train, y_train)
test_dataset = CustomerDataset(X_test, y_test)

# Create DataLoader instances
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True)



Creating NN model

In [ ]:
from torch import nn

# creating 4 layer RELU
class ModelV4(nn.Module):
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
        nn.Linear(in_features=124, out_features=10),
        nn.ReLU(),
        nn.Linear(in_features=10, out_features=10),
        nn.ReLU(),
        nn.Linear(in_features=10, out_features=10),
        nn.ReLU(),
        nn.Linear(in_features=10, out_features=10)
    )

  def forward(self, x: torch.Tensor) -> torch.Tensor:
      return self.layers(x)

modelv4 = ModelV4()
modelv4

ModelV4(
  (layers): Sequential(
    (0): Linear(in_features=124, out_features=10, bias=True)
    (1): ReLU()
    (2): Linear(in_features=10, out_features=10, bias=True)
    (3): ReLU()
    (4): Linear(in_features=10, out_features=10, bias=True)
    (5): ReLU()
    (6): Linear(in_features=10, out_features=10, bias=True)
  )
)

In [ ]:
!pip install torchinfo

In [ ]:
from torchinfo import summary

summary(modelv4, input_size=(32, 124))


Layer (type:depth-idx)                   Output Shape              Param #
ModelV4                                  [32, 10]                  --
├─Sequential: 1-1                        [32, 10]                  --
│    └─Linear: 2-1                       [32, 10]                  1,250
│    └─ReLU: 2-2                         [32, 10]                  --
│    └─Linear: 2-3                       [32, 10]                  110
│    └─ReLU: 2-4                         [32, 10]                  --
│    └─Linear: 2-5                       [32, 10]                  110
│    └─ReLU: 2-6                         [32, 10]                  --
│    └─Linear: 2-7                       [32, 10]                  110
Total params: 1,580
Trainable params: 1,580
Non-trainable params: 0
Total mult-adds (M): 0.05
Input size (MB): 0.02
Forward/backward pass size (MB): 0.01
Params size (MB): 0.01
Estimated Total Size (MB): 0.03

# Train Model

In [ ]:
!pip install tensorboard

In [ ]:
%load_ext tensorboard

## Creating training and test steps

In [ ]:
!pip install -U "ray[data,train,tune,serve]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 79.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 83.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 425.7/425.7 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.4/341

In [77]:
# Train step function
def train_step(model: torch.nn.Module,
               train_dataloader: torch.utils.data.DataLoader,
               loss: torch.nn.Module,
               optimizer: torch.optim.Optimizer):
  model.train()  # Set model to train mode

  train_loss, train_acc = 0, 0

  for batch, (X, y) in enumerate(train_dataloader):
      # Forward pass
      y_pred = model(X)

      # Calculate loss
      loss_val = loss(y_pred, y)
      train_loss += loss_val.item()

      # Optimizer zero grad
      optimizer.zero_grad()

      # Backward pass
      loss_val.backward()

      # Update weights
      optimizer.step()

      # Calculate accuracy
      y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
      train_acc += (y_pred_class == y).sum().item() / len(y_pred)

  # Average loss and accuracy per batch
  train_loss /= len(train_dataloader)
  train_acc /= len(train_dataloader)

  return train_loss, train_acc

# Test step function
def test_step(model: torch.nn.Module,
              test_dataloader: torch.utils.data.DataLoader,
              loss: torch.nn.Module):
  model.eval()  # Set model to evaluation mode

  test_loss, test_acc = 0, 0

  with torch.inference_mode():
      for batch, (X, y) in enumerate(test_dataloader):
          # Forward pass
          test_y_logits = model(X)

          # Calculate loss
          loss_val = loss(test_y_logits, y)
          test_loss += loss_val.item()

          # Calculate accuracy
          test_pred_labels = test_y_logits.argmax(dim=1)
          test_acc += (test_pred_labels == y).sum().item() / len(test_pred_labels)

  # Average loss and accuracy per batch
  test_loss /= len(test_dataloader)
  test_acc /= len(test_dataloader)

  return test_loss, test_acc

# Main training loop function
def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          loss: torch.nn.Module,
          optimizer: torch.optim.Optimizer,
          epochs: int):

  results = {
      "train_loss": [],
      "train_acc": [],
      "test_loss": [],
      "test_acc": []
  }

  for epoch in tqdm(range(epochs)):

      train_loss, train_acc = train_step(model=model,
                                          train_dataloader=train_dataloader,
                                          loss=loss,
                                          optimizer=optimizer)

      test_loss, test_acc = test_step(model=model,
                                      test_dataloader=test_dataloader,
                                      loss=loss)

      print(f"Epoch: {epoch+1} | Train Loss: {train_loss:.4f} | Test Loss: {test_loss:.4f} | Train Acc: {train_acc:.4f} | Test Acc: {test_acc:.4f}")

      # Append into results
      results["train_loss"].append(train_loss)
      results["train_acc"].append(train_acc)
      results["test_loss"].append(test_loss)
      results["test_acc"].append(test_acc)

  return results

In [ ]:
# Install mlflow and pyngrok (only needed the first time)
!pip install mlflow --quiet
!pip install pyngrok --quiet

In [79]:
import torch
import torch.nn as nn
from torch.optim import Adam
from tqdm import tqdm
from timeit import default_timer as timer
import mlflow


# Define model registry path and create it if it doesn't exist
#MODEL_REGISTRY = Path("./mlflow_experiments/")
#MODEL_REGISTRY.mkdir(exist_ok=True)

# Set tracking URI
#MLFLOW_TRACKING_URI = f"file://{MODEL_REGISTRY.absolute()}"
#mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

# Authentication for ngrok (replace with your actual authtoken)
!ngrok config add-authtoken 2mUbvf2qSsujKK9a6fKguc2yuCf_3k2z9A1WCHGGHDtAP4uqL

# Set tracking URI
#MLFLOW_TRACKING_URI = f"{MODEL_REGISTRY.absolute()}"
mlflow.set_tracking_uri("https://5f5e-34-168-89-14.ngrok-free.app")


# Authentication for ngrok (replace with your actual authtoken)
#ngrok.set_auth_token("2mUbvf2qSsujKK9a6fKguc2yuCf_3k2z9A1WCHGGHDtAP4uqL")


# Training Loop

NUM_EPOCHS = 2

loss = torch.nn.CrossEntropyLoss()
optimizer = Adam(modelv4.parameters(), lr=0.01)

start_time = timer()


with mlflow.start_run():
    # Training loop
    model_4_results = train(model=modelv4,
                            train_dataloader=train_dataloader,
                            test_dataloader=test_dataloader,
                            loss=loss,
                            optimizer=optimizer,
                            epochs=NUM_EPOCHS)

    # Log the trained model
    mlflow.pytorch.log_model(modelv4, "model2")

    mlflow.log_param("param1", 5)
    mlflow.log_metric("metric1", 0.86)


end_time = timer()
print(f"Total time: {end_time - start_time:.4f} seconds")

# Start MLflow server in the background
ngrok.kill()
get_ipython().system_raw("mlflow ui --port 5000 &")
ngrok_tunnel = ngrok.connect(5000)
print(f"MLflow Tracking UI: {ngrok_tunnel.public_url}")




Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


MlflowException: API request to endpoint /api/2.0/mlflow/runs/create failed with error code 404 != 200. Response body: '<!DOCTYPE html>
<html class="h-full" lang="en-US" dir="ltr">
  <head>
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-Regular-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-RegularItalic-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-Medium-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-Semibold-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-MediumItalic-WebS.woff" as="font" type="font/woff" c